In [1]:
import random
import torch
from sklearn.model_selection import KFold
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd


In [2]:

import torch
from torch import nn


class Net(nn.Module):
    def __init__(self, n_chans, n_classes):
        super(Net, self).__init__()
        self.temp_conv1 = nn.Conv2d(n_chans, n_chans, kernel_size=(1, 2),  groups=n_chans)
        self.temp_conv2 = nn.Conv2d(n_chans, n_chans, kernel_size=(1, 2),  groups=n_chans)
        self.temp_conv3 = nn.Conv2d(n_chans, n_chans, kernel_size=(1, 2),  groups=n_chans)
        self.temp_conv4 = nn.Conv2d(n_chans, n_chans, kernel_size=(1, 2),  groups=n_chans)
        self.temp_conv5 = nn.Conv2d(n_chans, n_chans, kernel_size=(1, 2),  groups=n_chans)
        self.temp_conv6 = nn.Conv2d(n_chans, n_chans, kernel_size=(1, 2),  groups=n_chans)

        self.chpool1 = nn.Sequential(
            nn.Conv2d(n_chans, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),  # 更改为 nn.BatchNorm2d
            
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),  # 更改为 nn.BatchNorm2d
            
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),  # 更改为 nn.BatchNorm2d
            
            nn.AdaptiveAvgPool2d((1, 1))  # 修改为 nn.AdaptiveAvgPool2d((1, 1))
        )

        # 其他的 chpool 代码保持不变...
        self.chpool2 = nn.Sequential(
            nn.Conv2d(n_chans, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.chpool3 = nn.Sequential(
            nn.Conv2d(n_chans, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.chpool4 = nn.Sequential(
            nn.Conv2d(n_chans, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.chpool5 = nn.Sequential(
            nn.Conv2d(n_chans, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=(1, 4), groups=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.classifier = nn.Sequential(
            nn.Linear(160, 64),
            #nn.LeakyReLU(0.01),
            nn.Linear(64, 32),
            #nn.Sigmoid(),
            nn.Linear(32, n_classes)
        )

    def forward(self, x, training=True):
        temp_x = self.temp_conv1(x)
        temp_w1 = self.temp_conv2(temp_x)
        temp_w2 = self.temp_conv3(temp_w1)
        temp_w3 = self.temp_conv4(temp_w2)
        temp_w4 = self.temp_conv5(temp_w3)
        temp_w5 = self.temp_conv6(temp_w4)

        w1 = self.chpool1(temp_w1)
        w2 = self.chpool2(temp_w2)
        w3 = self.chpool3(temp_w3)
        w4 = self.chpool4(temp_w4)
        w5 = self.chpool5(temp_w5)

        concat_vector = torch.cat([w1, w2, w3, w4, w5], 1)
        concat_vector = concat_vector.view(concat_vector.size(0), -1)  # Flatten
        classes = self.classifier(concat_vector)

        return classes

In [3]:
#导入数据
#rest1
datapath1=r'D:\JQ_YJS\飞行试验数据\处理后\rest1.npy' 
data1=np.load(datapath1)
#print(data1.shape)
data1=torch.tensor(data1).unsqueeze(1)
print(data1.shape)
data1=data1.numpy()

#rest2
datapath2=r'D:\JQ_YJS\飞行试验数据\处理后\rest2.npy'
data2=np.load(datapath2)
#print(data2.shape)
data2=torch.tensor(data2).unsqueeze(1)
print(data2.shape)
data2=data2.numpy()
data_all=np.concatenate((data1,data2),axis=0)
#标签制作
label_all = torch.cat([torch.zeros(6344), torch.ones(6589)]).long()  # 标签：前10505个为0，后10505个为1
print(label_all.unique())
print(data_all.shape)




torch.Size([6344, 1, 30, 200])
torch.Size([6589, 1, 30, 200])
tensor([0, 1])
(12933, 1, 30, 200)


In [4]:
# 创建十折交叉验证
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 保存结果的列表
historys = []
test_pred = []
test_real = []
accuracy, precision, recall, f1score = [], [], [], []
batchsz = 16
num_epochs = 100

# 进行十折交叉验证
for fold, (train_ind, test_ind) in enumerate(kfold.split(data_all, label_all)):
    print('fold号:', fold + 1)

   

    # 每一折验证前都要打乱训练集样本顺序
    n = len(train_ind)
    A = np.linspace(0, n - 1, n, dtype=int)
    random.shuffle(A)

    # 构建训练集、验证集、测试集
    epoch_train = data_all[train_ind[A[:int(0.8 * n)]]]
    epoch_val = data_all[train_ind[A[int(0.8 * n):]]]
    epoch_test = data_all[test_ind]
    label_train = label_all[train_ind[A[:int(0.8 * n)]]]
    label_val = label_all[train_ind[A[int(0.8 * n):]]]
    label_test = label_all[test_ind]

    # 转换为Tensor并创建DataLoader
    train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))
    
    train_loader = DataLoader(train_dataset, batch_size=batchsz, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsz, shuffle=False)

    # 选择、创建模型
    model = Net(n_chans=1,n_classes=1)
    #print(model)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # 配置模型训练
    criterion = nn.BCEWithLogitsLoss()  # 使用交叉熵损失
    
    optimizer = optim.SGD(model.parameters(), lr=1e-4)
    #optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

    # 开始训练模型
    print('开始训练!!')
    
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(num_epochs):  # 训练100个epoch
        model.train()
        running_loss = 0.0

        # 训练阶段
        for inputs, targets in train_loader:
            #print(f"训练批次输入数据形状: {inputs.shape}")
            inputs = inputs.to(device)
            
            targets = targets.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            targets = targets.view(-1).float() 
            #targets=targets.view(-1,1).float()
            loss = criterion(outputs.view(-1), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        #print('Epoch:{}/{}'.format(epoch+1,num_epochs))
        
        # 计算训练集的指标
        with torch.no_grad():
            train_pred = model(torch.tensor(epoch_train, dtype=torch.float32).to(device))
            train_pred = (torch.sigmoid(train_pred) > 0.5).cpu().numpy().astype(int)
            
            acc_train = accuracy_score(label_train, train_pred)
            pre_train = precision_score(label_train, train_pred, average='macro')
            rec_train = recall_score(label_train, train_pred, average='macro')
            f1_train = f1_score(label_train, train_pred, average='macro')

        # 输出训练集指标
       
        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                

                inputs = inputs.to(device)
            
                targets = targets.to(device)
                outputs = model(inputs)
                #targets=targets.view(-1,1).float()
                targets = targets.view(-1).float()
                
                loss = criterion(outputs.view(-1), targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        
        history['val_loss'].append(avg_val_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, VAL Loss: {avg_val_loss:.4f}')
        #print(f'Epoch [{epoch+1}/{num_epochs}], VAL Loss: {avg_val_loss:.4f}')
        print(f"$$训练集准确率 accuracy: {acc_train}",f"$$训练集精确率 precision: {pre_train}",f"$$训练集召回率 recall: {rec_train}",f"$$训练集 F1 评分 f1_score: {f1_train}")
        


        # 保存训练记录
        historys.append(history)
        

        # 计算、保存测试结果
        model.eval()
        with torch.no_grad():
            pred_test = model(torch.tensor(epoch_test, dtype=torch.float32).to(device))
            
            pred_test = (torch.sigmoid(pred_test) > 0.5).cpu().numpy().astype(int)


        # 保存预测结果和真实结果
        test_pred.append(pred_test)
        test_real.append(label_test)

        # 计算准确率，精确率，召回率，F1评分
        acc = accuracy_score(label_test, pred_test)
        pre = precision_score(label_test, pred_test, average='macro')
        rec = recall_score(label_test, pred_test, average='macro')
        f1 = f1_score(label_test, pred_test, average='macro')
        
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
        f1score.append(f1)
        
        print(f"@@测试集准确率 accuracy: {acc}",f"@@测试集精确率 precision: {pre}",f"@@测试集召回率 recall: {rec}",f"@@测试集 F1 评分 f1_score: {f1}")
        

# 将每一折 history 中误差结果保存（训练集和测试集，用于反映训练过程）    
loss_train = []
loss_val = []
for history_s in historys:
    loss_val.append(history_s['val_loss'])
    loss_train.append(history_s['train_loss'])




fold号: 1


C:\Users\FJQ\AppData\Local\Temp\ipykernel_21672\2547753516.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_21672\2547753516.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.6834, VAL Loss: 0.6821
$$训练集准确率 accuracy: 0.5563312211362904 $$训练集精确率 precision: 0.7043118266536008 $$训练集召回率 recall: 0.5645733899330555 $$训练集 F1 评分 f1_score: 0.46999314075782533
@@测试集准确率 accuracy: 0.5633693972179289 @@测试集精确率 precision: 0.6907947199318701 @@测试集召回率 recall: 0.5588780581039755 @@测试集 F1 评分 f1_score: 0.46921434488326286
Epoch [2/100], Train Loss: 0.6795, VAL Loss: 0.6832
$$训练集准确率 accuracy: 0.6019761572333798 $$训练集精确率 precision: 0.7262285344247901 $$训练集召回率 recall: 0.6091668342170578 $$训练集 F1 评分 f1_score: 0.5464834366786341
@@测试集准确率 accuracy: 0.5486862442040186 @@测试集精确率 precision: 0.6938786268556005 @@测试集召回率 recall: 0.5439339640672782 @@测试集 F1 评分 f1_score: 0.43716503061087697
Epoch [3/100], Train Loss: 0.6760, VAL Loss: 0.6781
$$训练集准确率 accuracy: 0.6401031038556546 $$训练集精确率 precision: 0.7430808799685655 $$训练集召回率 recall: 0.6464271794613315 $$训练集 F1 评分 f1_score: 0.6030709082983526
@@测试集准确率 accuracy: 0.6020092735703245 @@测试集精确率 precision: 0.7179

C:\Users\FJQ\AppData\Local\Temp\ipykernel_21672\2547753516.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_21672\2547753516.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!


...

d:\anaconda\envs\DL\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


@@测试集准确率 accuracy: 0.4911059551430781 @@测试集精确率 precision: 0.24555297757153904 @@测试集召回率 recall: 0.5 @@测试集 F1 评分 f1_score: 0.32935684647302904


d:\anaconda\envs\DL\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [15/100], Train Loss: 0.6519, VAL Loss: 0.6652
$$训练集准确率 accuracy: 0.5092353951890034 $$训练集精确率 precision: 0.2546176975945017 $$训练集召回率 recall: 0.5 $$训练集 F1 评分 f1_score: 0.33741283620321616
@@测试集准确率 accuracy: 0.4918793503480278 @@测试集精确率 precision: 0.7457430340557276 @@测试集召回率 recall: 0.5007598784194529 @@测试集 F1 评分 f1_score: 0.33104521404559284


KeyboardInterrupt: 

In [ ]:
# 假设 accuracy 列表包含了 1000 个准确率值
# 每一折包含 100 个 epoch
num_folds = 10
epochs_per_fold = 100

# 打印每一折的最大准确率和对应的epoch
for fold in range(num_folds):
    # 获取当前折的准确率列表
    start_idx = fold * epochs_per_fold
    end_idx = (fold + 1) * epochs_per_fold
    fold_accuracy = accuracy[start_idx:end_idx]

    # 找到最大准确率和对应的epoch
    max_acc = max(fold_accuracy)
    max_epoch = fold_accuracy.index(max_acc) + 1  # +1 因为索引从0开始

    # 打印结果
    print(f'第{fold + 1}折准确率最高为{max_acc:.4f}，对应的epoch为{max_epoch}')